# 테스트 스위트를 만들어두자! 

## ? 단위 테스트와 테스트를 혼동하지 말자
대게 테스트를 만들고 있다고 말하면, **단위테스트**를 만들고 있는것이다.


### 단위테스트란
한 구성요소 기능의 한 가지 측면만 테스트한다. 
(ex 은행계좌에서 인출할때 액수가 음수이면, 메시지와 함께 에러가 발생하는지 테스트)
- 테스트 구성요소에 따라 단위 테스트 같은 클래스에 그룹으로 묶을때도 많다. (음수인출액일때, 해당 계좌에서 어떤 행동을 취하는지 연속해서 테스트)
- 각 단위 테스트는 완벽히 독립적이어야 하며, 각 단위 테스트의 시작과 끝은 반드시 그 단위 테스트 자체에서 모두 이루어져야한다. 
- ! 중요 ! 각 단위 테스트에서는 보통 어썰션이 최소하나 들어간다. 어썰션이란 '단언'이란 뜻이고, 
- 2+2는 4라고 확정적으로 말하는것이다. 이 어서션은 2 + 2가 4임을 확정적으로 말하고, 조건이 참이면 테스트는 성공적으로 통과합니다. 조건이 거짓이면 테스트는 실패하고 오류가 발생합니다. 가끔 단위 테스트에 실패 상태만 포함될 때도 있습니다. 
- 예를 들어 예외가 발생하지 않은 상황을 실패상태로 정하면 예외가
발생하지 않았을 때는 아무 일도 하지 않고 넘어갈거다.

# 파이썬의 unittest 모듈 


In [4]:
# 2+2 = 4

import unittest



class TestAddition(unittest.TestCase):
    # 처음과 끝에서 동작하는 setUp, tearDown 매서드
    def setUp(self):
        print('Setting up the test')
    
    def tearDown(self):
        print('Tearing down the test')
    # test_ 로 시작하는 모든 함수를 단위 테스트로 실행하고 이 전치사가 없는 함수는 무시 
    def test_twoPlusTwo(self):
        total = 2 + 2
        self.assertEqual(4, total)

# 이건 파이썬에서 테스트를 실행하는 표준방법이다.
# if __name__ == '__main__':
#     unittest.main()

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)
    %reset # 꼭 리셋해야 TestCase를 상속하는 메서드를 메모리에서 비울 수 있다.(파이썬을 종료 안 시키는 설정때문에)


.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Setting up the test
Tearing down the test
Nothing done.


## 위키백과 테스트

In [8]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import unittest

class TestWikipedia(unittest.TestCase):
    bs = None
    
    # 매 테스트마다 실행되는 setUp과 달리 클래스를 시작할때 단 한번 실행된다. (즉 페이지를 한 번만 불러와서 여러 테스트를 실행한다.)
    # setUpClass 는 클래스 자체에 속하는 static 매서드이다. 전역 클래스 변수를 갖고 setUp은 클래스의 인스턴스에 속하는 인스턴스 함수라는 차이가 있다.
    def setUpClass():
        url = 'http://en.wikipedia.org/wiki/Monty_Python'
        TestWikipedia.bs = BeautifulSoup(urlopen(url), 'html.parser')

    def test_titleText(self):
        pageTitle = TestWikipedia.bs.find('h1').get_text()
        self.assertEqual('Monty Python', pageTitle);

    def test_contentExists(self):
        content = TestWikipedia.bs.find('div',{'id':'mw-content-text'})
        self.assertIsNotNone(content)


if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)
    %reset

.

Setting up the test
Tearing down the test


..
----------------------------------------------------------------------
Ran 3 tests in 1.053s

OK


Nothing done.


## static method (파이썬에서는 class Method라고 하네) 와 인스턴스 매서드 확실히 구분하세요
 setUpClass(class method)와 setUp(instance method)의 차이를 확실히 알아두세요.
 

In [9]:
# 각 페이지마다 어썰션 실행하는 단위 테스트
from urllib.request import urlopen
from bs4 import BeautifulSoup
import unittest
import re
import random
from urllib.parse import unquote

class TestWikipedia(unittest.TestCase):

    def test_PageProperties(self):
        self.url = 'http://en.wikipedia.org/wiki/Monty_Python'
        #Test the first 10 pages we encounter
        for i in range(1, 10):
            self.bs = BeautifulSoup(urlopen(self.url), 'html.parser')
            titles = self.titleMatchesURL()
            self.assertEqual(titles[0], titles[1])
            self.assertTrue(self.contentExists())
            self.url = self.getNextLink()
        print('Done!')

    def titleMatchesURL(self):
        pageTitle = self.bs.find('h1').get_text()
        urlTitle = self.url[(self.url.index('/wiki/')+6):]
        urlTitle = urlTitle.replace('_', ' ')
        urlTitle = unquote(urlTitle)
        return [pageTitle.lower(), urlTitle.lower()]

    def contentExists(self):
        content = self.bs.find('div',{'id':'mw-content-text'})
        if content is not None:
            return True
        return False

    def getNextLink(self):
        # Returns random link on page, using technique from Chapter 3
        links = self.bs.find('div', {'id':'bodyContent'}).find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))
        randomLink = random.SystemRandom().choice(links)
        return f'https://wikipedia.org{randomLink.attrs["href"]}'


if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)
    %reset

.

Setting up the test
Tearing down the test


.
----------------------------------------------------------------------
Ran 2 tests in 17.167s

OK


Done!
Nothing done.


# Selenium을 이용한 웹테스트
## 문법이 많이 다르다. 
셀레니움 단위 테스트는 클래스 안에 함수로 저장하지 않아도 됩니다. 


In [11]:
from selenium import webdriver
import chromedriver_autoinstaller 

chromedriver_autoinstaller.install() # 크롬드라이버 자동 설치
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)
driver.get('http://www.google.com')
# 괄호 필요없고, assert를 사용하여 테스트를 실행한다.
assert "Monty Python" in driver.title
driver.close()


AssertionError: 

## 근데 이 폼의 기능이 동작한다는 보장이 있나?? 폼의 기능을 테스트하고 브라우저에서 동작하는지 확인하려면?? 


In [ ]:
# 객체가 반환됨 
usernameField = driver.find_element(By.NAME, 'username')

# 셀레니움도 요소에 다양한 행동을 취할 수 있음.

myElement.click()
myElement.click_and_hold()
myElement.release()
myElement.double_click()
myElement.send_keys_to_element('Hello, World!')


이러한 행동을 하나의 체인으로 묶어서 저장하고 원하는 만큼 실행시키기 가능. (action chain)


In [14]:
from prompt_toolkit.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import chromedriver_autoinstaller
from selenium import webdriver

chromedriver_autoinstaller.install()

options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)
driver.get('http://pythonscraping.com/pages/files/form.html')

firstnameField = driver.find_element(By.NAME, 'firstname')
lastnameField = driver.find_element(By.NAME, 'lastname')
submitButton = driver.find_element(By.ID, 'submit')

# 방법 1
# firstnameField.send_keys('Ryan')
# lastnameField.send_keys('Mitchell')
# submitButton.click()
# ##
# 방법 2
actions = ActionChains(driver).click(firstnameField).send_keys('Ryan').click(lastnameField).send_keys('Mitchell').send_keys(Keys.RETURN)
actions.perform() # perform()을 호출했을때 차례로 일어난다.  두번째 방법은 엔터를 누른거임 ( 첫번째 방법은 버튼을 클릭한거고 ) 
## 
print(driver.find_element(By.TAG_NAME, 'body').text)\

driver.close()

AttributeError: RETURN

## 드래그 앤 드롭까지 가능 ㄷㄷ 


In [ ]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import unittest


class TestDragAndDrop(unittest.TestCase):
    driver = None

    def setUp(self):
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        self.driver = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH), options=chrome_options)
        url = 'http://pythonscraping.com/pages/javascript/draggableDemo.html'
        self.driver.get(url)

    def tearDown(self):
        self.driver.close()

    def test_drag(self):
        element = self.driver.find_element(By.ID, 'draggable')
        target = self.driver.find_element(By.ID, 'div2')
        actions = ActionChains(self.driver)
        actions.drag_and_drop(element, target).perform()
        self.assertEqual('You are definitely not a bot!', self.driver.find_element(By.ID, 'message').text)

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)
    %reset

## 스크린샷 찍기 
감동이네 감동 


In [16]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(options=options)

driver.get('http://www.pythonscraping.com/')
driver.get_screenshot_as_file('./pythonscraping.png')


True

# 그래서 unittest vs selenium 
